In [1]:
## A tutorial script for well log prediction 
# This tutorial is for the most general case:
# inputing 4 values from four existing well logs and predicting the missing S-wave log (DT)
# 
# INPUT: P-wave sonic log (DTCO), Gamma-ray (ECGR), Density (RHOB), Total porosity (PHIT)
# OUTPUT: S-wave sonic log (DTSM)

# first download the data from
!svn co https://github.com/chenyk1990/mldata/trunk/well ./well

Restored 'well/log4tutorial.xlsx'
Checked out revision 52.


In [2]:
!cp well/log4tutorial.xlsx ./

In [3]:
# Here we introduce a self-attention BiLSTM network for training a model

import numpy as np
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.layers import Activation, Add, Bidirectional, Conv1D, Dense, Dropout, Embedding, Flatten, Reshape, multiply
from keras.layers import concatenate, GRU, Input, LSTM, MaxPooling1D
from keras.layers import GlobalAveragePooling1D,  GlobalMaxPooling1D, SpatialDropout1D
from keras.models import Model
from keras.preprocessing import text, sequence
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss
from sklearn.model_selection import train_test_split
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.models import Model

def SE_Block(tensor, ratio=8):
    init = tensor
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    filters = int(init.shape[-1])
    se_shape = (1, filters)
    se = GlobalAveragePooling1D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    x = multiply([init, se])
    x = Add()([x,init])
    return x

inp1 = layers.Input(shape=(4,1),name='input_layer')
D1 = 8
D2 = int(2*D1)
D3 = int(2*D2)
D4 = int(2*D3)
D5 = int(2*D4)

features = Bidirectional(LSTM(D1, return_sequences=True))(inp1)
features = SE_Block(features)
features = Bidirectional(LSTM(D2, return_sequences=True))(features)
features = SE_Block(features)
features = Bidirectional(LSTM(D3, return_sequences=True))(features)
features = SE_Block(features)
features = Bidirectional(LSTM(D4, return_sequences=False))(features)

#features = Dense(D4,activation='relu')(features)
#features = Dense(D3,activation='relu')(features)
#features = Dense(D2,activation='relu')(features)

e = Dense(1)(features)
o = Activation('linear', name='output_layer')(e)
model = Model(inputs=[inp1], outputs=o)
model.summary()



2023-04-06 22:22:42.195391: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 22:22:46.931598: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_layer (InputLayer)       [(None, 4, 1)]       0           []                               
                                                                                                  
 bidirectional (Bidirectional)  (None, 4, 16)        640         ['input_layer[0][0]']            
                                                                                                  
 global_average_pooling1d (Glob  (None, 16)          0           ['bidirectional[0][0]']          
 alAveragePooling1D)                                                                              
                                                                                                  
 reshape (Reshape)              (None, 1, 16)        0           ['global_average_pooling1d[0]

In [4]:
# Bebig the training

import pandas as pd
import tensorflow.keras
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import keras
sgd = tensorflow.keras.optimizers.Adam(lr=1e-3)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                                cooldown= 0,
                                patience= 50,
                                min_lr=0.5e-8)

early_stopping_monitor = EarlyStopping(monitor= 'val_loss', patience = 100)

for fname in ['log4tutorial.xlsx']:
    df = pd.read_excel(fname)
    df_raw_input = df[['DTCO', 'ECGR', 'RHOB', 'PHIT']]
    X = df_raw_input
    y = df['DTSM']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    checkpoint = ModelCheckpoint(filepath='best_model_%s.h5'%(fname.split('.')[0]),
                             monitor='val_loss',
                             mode = 'auto',
                             verbose=1,
                             save_best_only=True)
    callbacks = [lr_reducer, early_stopping_monitor, checkpoint]
    model = Model(inputs=[inp1], outputs=o)
    model.compile(optimizer=sgd, loss='mse', metrics=['mse'])
    history = model.fit([X_train], y_train, epochs=200, validation_data=([X_test],y_test), shuffle=True, batch_size=128, callbacks = callbacks)


/Users/chenyk/opt/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
346/349 [============================>.] - ETA: 0s - loss: 6288.6826 - mse: 6288.6826
Epoch 1: val_loss improved from inf to 3165.15283, saving model to best_model_log4tutorial.h5
349/349 [==============================] - 17s 21ms/step - loss: 6267.0356 - mse: 6267.0356 - val_loss: 3165.1528 - val_mse: 3165.1528 - lr: 0.0010
Epoch 2/200
347/349 [============================>.] - ETA: 0s - loss: 1830.0050 - mse: 1830.0050
Epoch 2: val_loss improved from 3165.15283 to 888.92865, saving model to best_model_log4tutorial.h5
349/349 [==============================] - 7s 20ms/step - loss: 1826.0298 - mse: 1826.0298 - val_loss: 888.9286 - val_mse: 888.9286 - lr: 0.0010
Epoch 3/200
347/349 [============================>.] - ETA: 0s - loss: 541.7598 - mse: 541.7598
Epoch 3: val_loss improved from 888.92865 to 320.88644, saving model to best_model_log4tutorial.h5
349/349 [==============================] - 7s 19ms/step - loss: 540.6859 - mse: 540.6859 - val_loss: 320.8864 - val_mse: 3

Epoch 27/200
349/349 [==============================] - ETA: 0s - loss: 30.7486 - mse: 30.7486
Epoch 27: val_loss did not improve from 28.96081
349/349 [==============================] - 9s 26ms/step - loss: 30.7486 - mse: 30.7486 - val_loss: 29.1100 - val_mse: 29.1100 - lr: 0.0010
Epoch 28/200
347/349 [============================>.] - ETA: 0s - loss: 29.6868 - mse: 29.6868
Epoch 28: val_loss improved from 28.96081 to 28.41645, saving model to best_model_log4tutorial.h5
349/349 [==============================] - 8s 21ms/step - loss: 29.7947 - mse: 29.7947 - val_loss: 28.4164 - val_mse: 28.4164 - lr: 0.0010
Epoch 29/200
347/349 [============================>.] - ETA: 0s - loss: 28.9081 - mse: 28.9081
Epoch 29: val_loss improved from 28.41645 to 27.49255, saving model to best_model_log4tutorial.h5
349/349 [==============================] - 9s 26ms/step - loss: 28.8994 - mse: 28.8994 - val_loss: 27.4925 - val_mse: 27.4925 - lr: 0.0010
Epoch 30/200
349/349 [==============================]

348/349 [============================>.] - ETA: 0s - loss: 25.5605 - mse: 25.5605
Epoch 54: val_loss did not improve from 24.57892
349/349 [==============================] - 13s 37ms/step - loss: 25.5572 - mse: 25.5572 - val_loss: 25.1644 - val_mse: 25.1644 - lr: 0.0010
Epoch 55/200
348/349 [============================>.] - ETA: 0s - loss: 25.4436 - mse: 25.4436
Epoch 55: val_loss did not improve from 24.57892
349/349 [==============================] - 10s 30ms/step - loss: 25.4232 - mse: 25.4232 - val_loss: 24.5984 - val_mse: 24.5984 - lr: 0.0010
Epoch 56/200
348/349 [============================>.] - ETA: 0s - loss: 25.3604 - mse: 25.3604
Epoch 56: val_loss did not improve from 24.57892
349/349 [==============================] - 12s 36ms/step - loss: 25.3519 - mse: 25.3519 - val_loss: 27.9582 - val_mse: 27.9582 - lr: 0.0010
Epoch 57/200
347/349 [============================>.] - ETA: 0s - loss: 26.4418 - mse: 26.4418
Epoch 57: val_loss did not improve from 24.57892
349/349 [========

Epoch 82/200
348/349 [============================>.] - ETA: 0s - loss: 24.2100 - mse: 24.2100
Epoch 82: val_loss did not improve from 23.41517
349/349 [==============================] - 13s 36ms/step - loss: 24.1988 - mse: 24.1988 - val_loss: 24.4282 - val_mse: 24.4282 - lr: 0.0010
Epoch 83/200
347/349 [============================>.] - ETA: 0s - loss: 24.8916 - mse: 24.8916
Epoch 83: val_loss did not improve from 23.41517
349/349 [==============================] - 13s 37ms/step - loss: 24.8605 - mse: 24.8605 - val_loss: 26.5323 - val_mse: 26.5323 - lr: 0.0010
Epoch 84/200
347/349 [============================>.] - ETA: 0s - loss: 23.9583 - mse: 23.9583
Epoch 84: val_loss did not improve from 23.41517
349/349 [==============================] - 14s 41ms/step - loss: 24.0487 - mse: 24.0487 - val_loss: 23.9174 - val_mse: 23.9174 - lr: 0.0010
Epoch 85/200
348/349 [============================>.] - ETA: 0s - loss: 24.3710 - mse: 24.3710
Epoch 85: val_loss improved from 23.41517 to 23.40747

348/349 [============================>.] - ETA: 0s - loss: 23.9758 - mse: 23.9758
Epoch 110: val_loss did not improve from 22.97460
349/349 [==============================] - 11s 32ms/step - loss: 23.9673 - mse: 23.9673 - val_loss: 24.4712 - val_mse: 24.4712 - lr: 0.0010
Epoch 111/200
347/349 [============================>.] - ETA: 0s - loss: 23.5774 - mse: 23.5774
Epoch 111: val_loss did not improve from 22.97460
349/349 [==============================] - 12s 34ms/step - loss: 23.5703 - mse: 23.5703 - val_loss: 24.3840 - val_mse: 24.3840 - lr: 0.0010
Epoch 112/200
349/349 [==============================] - ETA: 0s - loss: 23.5341 - mse: 23.5341
Epoch 112: val_loss did not improve from 22.97460
349/349 [==============================] - 12s 35ms/step - loss: 23.5341 - mse: 23.5341 - val_loss: 23.0420 - val_mse: 23.0420 - lr: 0.0010
Epoch 113/200
348/349 [============================>.] - ETA: 0s - loss: 23.4154 - mse: 23.4154
Epoch 113: val_loss did not improve from 22.97460
349/349 [=

349/349 [==============================] - ETA: 0s - loss: 22.8887 - mse: 22.8887
Epoch 138: val_loss did not improve from 22.39085
349/349 [==============================] - 6s 19ms/step - loss: 22.8887 - mse: 22.8887 - val_loss: 23.3770 - val_mse: 23.3770 - lr: 0.0010
Epoch 139/200
349/349 [==============================] - ETA: 0s - loss: 23.1544 - mse: 23.1544
Epoch 139: val_loss did not improve from 22.39085
349/349 [==============================] - 6s 18ms/step - loss: 23.1544 - mse: 23.1544 - val_loss: 22.4309 - val_mse: 22.4309 - lr: 0.0010
Epoch 140/200
348/349 [============================>.] - ETA: 0s - loss: 22.9970 - mse: 22.9970
Epoch 140: val_loss did not improve from 22.39085
349/349 [==============================] - 11s 31ms/step - loss: 23.0309 - mse: 23.0309 - val_loss: 23.0311 - val_mse: 23.0311 - lr: 0.0010
Epoch 141/200
346/349 [============================>.] - ETA: 0s - loss: 23.2140 - mse: 23.2140
Epoch 141: val_loss did not improve from 22.39085
349/349 [===

349/349 [==============================] - ETA: 0s - loss: 23.0540 - mse: 23.0540
Epoch 166: val_loss did not improve from 22.00205
349/349 [==============================] - 9s 26ms/step - loss: 23.0540 - mse: 23.0540 - val_loss: 22.4595 - val_mse: 22.4595 - lr: 0.0010
Epoch 167/200
346/349 [============================>.] - ETA: 0s - loss: 22.1520 - mse: 22.1520
Epoch 167: val_loss did not improve from 22.00205
349/349 [==============================] - 9s 26ms/step - loss: 22.1728 - mse: 22.1728 - val_loss: 22.0898 - val_mse: 22.0898 - lr: 0.0010
Epoch 168/200
347/349 [============================>.] - ETA: 0s - loss: 22.4189 - mse: 22.4189
Epoch 168: val_loss did not improve from 22.00205
349/349 [==============================] - 8s 23ms/step - loss: 22.4160 - mse: 22.4160 - val_loss: 22.1667 - val_mse: 22.1667 - lr: 0.0010
Epoch 169/200
348/349 [============================>.] - ETA: 0s - loss: 21.9722 - mse: 21.9722
Epoch 169: val_loss did not improve from 22.00205
349/349 [====

349/349 [==============================] - 10s 29ms/step - loss: 22.2304 - mse: 22.2304 - val_loss: 22.1998 - val_mse: 22.1998 - lr: 0.0010
Epoch 195/200
347/349 [============================>.] - ETA: 0s - loss: 21.8947 - mse: 21.8947
Epoch 195: val_loss did not improve from 21.52619
349/349 [==============================] - 9s 27ms/step - loss: 21.8886 - mse: 21.8886 - val_loss: 22.7632 - val_mse: 22.7632 - lr: 0.0010
Epoch 196/200
347/349 [============================>.] - ETA: 0s - loss: 21.8754 - mse: 21.8754
Epoch 196: val_loss did not improve from 21.52619
349/349 [==============================] - 12s 34ms/step - loss: 21.8747 - mse: 21.8747 - val_loss: 23.1713 - val_mse: 23.1713 - lr: 0.0010
Epoch 197/200
348/349 [============================>.] - ETA: 0s - loss: 22.7242 - mse: 22.7242
Epoch 197: val_loss did not improve from 21.52619
349/349 [==============================] - 13s 37ms/step - loss: 22.7153 - mse: 22.7153 - val_loss: 22.7634 - val_mse: 22.7634 - lr: 0.0010
Epo

In [5]:
## Below is to test
import pandas as pd
from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

for fname in ['log4tutorial.xlsx']:
    df = pd.read_excel(fname)

    df_raw_input = df[['DTCO', 'ECGR', 'RHOB', 'PHIT']]
    X = df_raw_input
    y = df['DTSM']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

    model = load_model('best_model_%s.h5'%(fname.split('.')[0].split('_')[-1])) #current one:  0.8361
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    print(fname.split('.')[0]+' R2:', r2,' RMSE:',rmse)


598/598 [==============================] - 5s 4ms/step
log4tutorial R2: 0.9059577804227912  RMSE: 4.639632443577757


In [6]:
## Comparison with random forest
import pandas as pd
import numpy as np
from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
for fname in ['log4tutorial.xlsx']:
    df = pd.read_excel(fname)

    df_raw_input = df[['DTCO', 'ECGR', 'RHOB', 'PHIT']]
    X = df_raw_input
    y = df['DTSM']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

model = RandomForestRegressor(max_depth=5, random_state=0, n_estimators=5)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print(fname.split('.')[0]+' R2:', r2,' RMSE:',rmse)


log4tutorial R2: 0.8929411242623861  RMSE: 4.950322357831745
